In [17]:
import math
import random
import string
import numpy as np

In [18]:
random.seed(0)

In [19]:
# 生成区间[a, b)内的随机数
def rand(a, b):
    return (b - a) * random.random() + a

In [20]:
# 生成大小 I*J 的矩阵，默认零矩阵 
def makeMatrix(I, J, fill=0.0):
    m = []
    for i in range(I):
        m.append([fill] * J)
    return m

In [21]:
def sigmoid(x):
    # our activation function: f(x) = 1 / (1 * e^(-x))
    return 1 / (1 + np.exp(-x))

In [22]:
# 函数 sigmoid 的派生函数, 为了得到输出 (即：y)
def dsigmoid(y):
    # return 1.0 - y ** 2
    return y*(1-y)

In [23]:
class NN:
    ''' 三层反向传播神经网络 '''

    def __init__(self, ni, nh, no):
        # 输入层、隐藏层、输出层的节点（数）
        self.ni = ni + 1  # 增加一个偏差节点
        self.nh = nh
        self.no = no

        # 激活神经网络的所有节点（向量）
        self.ai = [1.0] * self.ni
        self.ah = [1.0] * self.nh
        self.ao = [1.0] * self.no

        # 建立权重（矩阵）
        self.wi = makeMatrix(self.ni, self.nh)
        self.wo = makeMatrix(self.nh, self.no)
        # 设为随机值
        for i in range(self.ni):
            for j in range(self.nh):
                self.wi[i][j] = rand(-0.2, 0.2)
        for j in range(self.nh):
            for k in range(self.no):
                self.wo[j][k] = rand(-2.0, 2.0)

        # 最后建立动量因子（矩阵）
        self.ci = makeMatrix(self.ni, self.nh)
        self.co = makeMatrix(self.nh, self.no)

    def update(self, inputs):
        if len(inputs) != self.ni - 1:
            raise ValueError('与输入层节点数不符！')

        # 激活输入层
        for i in range(self.ni - 1):
            # self.ai[i] = sigmoid(inputs[i])
            self.ai[i] = inputs[i]

        # 激活隐藏层
        for j in range(self.nh):
            sum = 0.0
            for i in range(self.ni):
                sum = sum + self.ai[i] * self.wi[i][j]
            self.ah[j] = sigmoid(sum)

        # 激活输出层
        for k in range(self.no):
            sum = 0.0
            for j in range(self.nh):
                sum = sum + self.ah[j] * self.wo[j][k]
            self.ao[k] = sigmoid(sum)

        return self.ao[:]

    def backPropagate(self, targets, N, M):
        ''' 反向传播 '''
        if len(targets) != self.no:
            raise ValueError('与输出层节点数不符！')

        # 计算输出层的误差
        output_deltas = [0.0] * self.no
        for k in range(self.no):
            error = targets[k] - self.ao[k]
            output_deltas[k] = dsigmoid(self.ao[k]) * error

        # 计算隐藏层的误差
        hidden_deltas = [0.0] * self.nh
        for j in range(self.nh):
            error = 0.0
            for k in range(self.no):
                error = error + output_deltas[k] * self.wo[j][k]
            hidden_deltas[j] = dsigmoid(self.ah[j]) * error

        # 更新输出层权重
        for j in range(self.nh):
            for k in range(self.no):
                change = output_deltas[k] * self.ah[j]
                self.wo[j][k] = self.wo[j][k] + N * change + M * self.co[j][k]
                self.co[j][k] = change
                # print(N*change, M*self.co[j][k])

        # 更新输入层权重
        for i in range(self.ni):
            for j in range(self.nh):
                change = hidden_deltas[j] * self.ai[i]
                self.wi[i][j] = self.wi[i][j] + N * change + M * self.ci[i][j]
                self.ci[i][j] = change

        # 计算误差
        error = 0.0
        for k in range(len(targets)):
            error = error + 0.5 * (targets[k] - self.ao[k]) ** 2
        return error

    def test(self, patterns):
        for p in patterns:
            print(p[0], '->', self.update(p[0]))

    def weights(self):
        print('输入层权重:')
        for i in range(self.ni):
            print(self.wi[i])
        print()
        print('输出层权重:')
        for j in range(self.nh):
            print(self.wo[j])

    def train(self, patterns, iterations=20000, N=0.2, M=0.5):
        # N: 学习速率(learning rate)
        # M: 动量因子(momentum factor)
        for i in range(iterations):
            error = 0.0
            for p in patterns:
                inputs = p[0]
                targets = p[1]
                self.update(inputs)
                error = error + self.backPropagate(targets, N, M)
            if i % 100 == 0:
                print('误差 %-.5f' % error)




In [24]:
def demo():
    pat = [
        [[0, 0], [0]],
        [[0, 1], [1]],
        [[1, 0], [1]],
        [[1, 1], [0]]
    ]

    # 创建一个神经网络：输入层有两个节点、隐藏层有两个节点、输出层有一个节点
    n = NN(2, 20, 1)
    # 训练
    n.train(pat)
    # 测试
    n.test(pat)
    # 训练的权重
    # n.weights()



if __name__ == '__main__':
    demo()

误差 0.88325
误差 0.62736
误差 0.57617
误差 0.48899
误差 0.28403
误差 0.09182
误差 0.03683
误差 0.02034
误差 0.01337
误差 0.00972
误差 0.00752
误差 0.00608
误差 0.00507
误差 0.00433
误差 0.00377
误差 0.00333
误差 0.00297
误差 0.00268
误差 0.00244
误差 0.00223
误差 0.00206
误差 0.00191
误差 0.00177
误差 0.00166
误差 0.00156
误差 0.00146
误差 0.00138
误差 0.00131
误差 0.00124
误差 0.00118
误差 0.00113
误差 0.00108
误差 0.00103
误差 0.00099
误差 0.00095
误差 0.00091
误差 0.00088
误差 0.00085
误差 0.00082
误差 0.00079
误差 0.00076
误差 0.00074
误差 0.00071
误差 0.00069
误差 0.00067
误差 0.00065
误差 0.00064
误差 0.00062
误差 0.00060
误差 0.00059
误差 0.00057
误差 0.00056
误差 0.00054
误差 0.00053
误差 0.00052
误差 0.00051
误差 0.00049
误差 0.00048
误差 0.00047
误差 0.00046
误差 0.00045
误差 0.00044
误差 0.00044
误差 0.00043
误差 0.00042
误差 0.00041
误差 0.00040
误差 0.00040
误差 0.00039
误差 0.00038
误差 0.00037
误差 0.00037
误差 0.00036
误差 0.00036
误差 0.00035
误差 0.00034
误差 0.00034
误差 0.00033
误差 0.00033
误差 0.00032
误差 0.00032
误差 0.00031
误差 0.00031
误差 0.00030
误差 0.00030
误差 0.00030
误差 0.00029
误差 0.00029
误差 0.00028
误差 0.00028
误差 0.00028